# Rag 入门
主要步骤：数据采集-> 分块处理 -> 嵌入 -> 语义检索 -> 响应生成

In [9]:
import fitz
import os
import numpy as np
import google.generativeai as genai
from dotenv import load_dotenv
import json

load_dotenv()

True

# 从PDF文件中提取文本

In [10]:
def extract_text_from_pdf(pdf_path):
    """
    提取pdf文件，并打印章节信息
    参数：
    pdf_path : pdf文件路径
    返回：
    str : 提取的文本结果
    """
    pdf_file = fitz.open(pdf_path)
    all_text = ""

    for n in range(pdf_file.page_count):
        page = pdf_file[n]
        text = page.get_text("text")
        all_text += text

    return all_text

In [11]:
result = extract_text_from_pdf("../data/AI_Information.en.zh-CN.pdf")
if result:
    print(f"pdf读取成功，共计{len(result)}字符")

pdf读取成功，共计10071字符


# 对提取的文本分块

In [12]:
def chunk_text(text, n, overlap):
    """
    对提取的文档结果进行分块
    参数：
    text : 返回的文本参数
    n : 块长度
    overlap : 分块之间重叠的大小
    """
    chunks = []

    for i in range(0, len(text), n-overlap):
        chunks.append(text[i:i+n])

    return chunks

In [14]:
text_chunks = chunk_text(result, 300, 10)
if text_chunks:
    print(f"分块成功，共计{len(text_chunks)}块")

分块成功，共计35块


# 设置API接口

In [15]:
genai.configure(
    api_key = os.getenv("API_KEY"),
    transport="rest"
)

# 文本块创建嵌入

In [16]:
def create_embeddings(text):
    response = genai.embed_content(
        model = os.getenv("EMBEDDING_MODEL"),
        content = text,
        task_type = "RETRIEVAL_DOCUMENT"
    )
    return response

response = create_embeddings(text_chunks)

In [17]:
len(response['embedding']), response

(35,
 {'embedding': [[-0.0037181717,
    0.01800313,
    0.019739946,
    -0.06133283,
    -0.01287024,
    0.013348926,
    -0.007823862,
    0.011538165,
    0.018595202,
    0.01156444,
    -0.012892349,
    -0.005661329,
    -0.019840876,
    0.012034518,
    0.1145596,
    0.00048122497,
    0.0074609406,
    0.0071967547,
    0.01379674,
    -0.026564967,
    0.014993546,
    -0.0055058277,
    -0.012881122,
    -0.00024884375,
    -0.02488215,
    0.0011731103,
    0.010346671,
    0.0087798275,
    0.0294322,
    -0.031916242,
    0.003050248,
    0.022861589,
    0.0063649323,
    0.014001472,
    -0.0058450606,
    0.011556524,
    0.011953234,
    -0.010010483,
    0.004216544,
    0.019496169,
    -0.006156168,
    -0.020005884,
    -0.023905002,
    -0.0002558616,
    0.015892176,
    -0.00021806953,
    0.018298268,
    -0.027579932,
    -0.020721711,
    0.0068532354,
    0.0027712837,
    0.011032089,
    -0.009078693,
    -0.2173733,
    0.0033778513,
    -0.0045574275

# 语义搜索

In [18]:
def cos_similarity(vec1, vec2):
    """
    使用余弦相似计算距离
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [19]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    对提问分块检索
    """
    query_embedding = create_embeddings(query)
    similarity_scores = []
    # print(embeddings)
    
    for i, chunk_embedding in enumerate(embeddings):
        # print(query_embedding['embedding'])
        similarity_score = cos_similarity(np.array(query_embedding['embedding']), np.array(chunk_embedding))
        # print(np.array(query_embedding['embedding'][0]), "\n", np.array(chunk_embedding))
        # break
        similarity_scores.append((i, similarity_score))

    # 按照分数进行排序
    
    similarity_scores.sort(key = lambda x : x[1], reverse=True)
    # 保留k个最接近的结果
    top_indices = [index for index, _ in similarity_scores[:k]]

    return [text_chunks[index] for index in top_indices]

In [20]:
# Load the validation data from a JSON file
with open('../data/val.json', encoding="utf-8") as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[0]['question']

# Perform semantic search to find the top 2 most relevant text chunks for the query
top_chunks = semantic_search(query, text_chunks, response['embedding'], k=2)

# Print the query
print("Query:", query)

# Print the top 2 most relevant text chunks
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

Query: 什么是‘可解释人工智能’，为什么它被认为很重要？
Context 1:
XAI 技术正在开发中，旨在深⼊了解⼈
⼯智能模型的决策⽅式，从⽽增强信任度和责任感。
边缘⼈⼯智能
边缘⼈⼯智能是指在设备上本地处理数据，⽽不是依赖云服务器。这种⽅法可以减少延迟，增强隐
私保护，并在连接受限的环境中⽀持⼈⼯智能应⽤。
量⼦计算和⼈⼯智能
量⼦计算有望显著加速⼈⼯智能算法，从⽽推动药物研发、材料科学和优化等领域的突破。量⼦计
算与⼈⼯智能的交叉研究前景⼴阔。
⼈机协作
⼈⼯智能的未来很可能涉及⼈类与⼈⼯智能系统之间更紧密的协作。这包括开发能够增强⼈类能
⼒、⽀持决策和提⾼⽣产⼒的⼈⼯智能⼯具。
⼈⼯智能造福社会
⼈⼯智能正⽇益被⽤于应对社会和环境挑战，例如⽓候变化、贫困和医疗
Context 2:
社会影响
⼈⼯智能的快速发展和部署引发了重⼤的伦理和社会担忧。这些担忧包括：
偏⻅与公平
⼈⼯智能系统可能会继承并放⼤其训练数据中存在的偏⻅，从⽽导致不公平或歧视性的结果。确保
⼈⼯智能系统的公平性并减少偏⻅是⼀项关键挑战。
透明度和可解释性
许多⼈⼯智能系统，尤其是深度学习模型，都是“⿊匣⼦”，很难理解它们是如何做出决策的。增
强透明度和可解释性对于建⽴信任和问责⾄关重要。
隐私和安全
⼈⼯智能系统通常依赖⼤量数据，这引发了⼈们对隐私和数据安全的担忧。保护敏感信息并确保负
责任的数据处理⾄关重要。
⼯作岗位流失
⼈⼯智能的⾃动化能⼒引发了⼈们对⼯作岗位流失的担忧，尤其是在重复性或常规性任务的
